# Imports And Libraries

In [1]:
from square.client import Client
import os
import pandas as pd

from google_sheets import *

# import pandas as pd
from boto import get_square_secret

# import os
import pickle

# Generic Functions

In [2]:
def get_value(key1,key2):
    try:
        return key1[key2] 
    except:
        return None

# Get Existing Subs

In [6]:
SAMPLE_SPREADSHEET_ID_input = '1K_uXqI1eiguBMr0uH3Tu7xDzePAIi3uAsDrAWukyBq8'
SAMPLE_RANGE_NAME = 'Square Subs!A:G'

exist = get_spread_sheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME)


# Get Data of most Recent Sub

In [7]:
dates = exist[exist['Date'].str.contains('T')]

most_recent_transaction_date = pd.to_datetime(dates['Date'], format='mixed').max().isoformat()

most_recent_transaction_date

'2023-10-19T10:06:28+00:00'

# Get New Subs

In [8]:
client = Client(
    access_token=get_square_secret()['SQUARE_ACCESS_TOKEN'],
    environment='production')


In [9]:
orders_api = client.orders
customers_api = client.customers

In [10]:
most_recent_transaction_date

'2023-10-19T10:06:28+00:00'

In [11]:
body = {
    'location_ids': [
        'SBBJSGR24YHYB'
    ],
    'query': {
        'filter': {
            
            'date_time_filter': {
                'updated_at': {
                    'start_at': f'2023-09-06T20:15:23+00:00',
                }
            },
      # "state_filter": {
        # "states": [
        #   "COMPLETED"
        # ]
      # }
       },
    "sort": {
      "sort_field": "UPDATED_AT",
      "sort_order": "DESC"
    }
        
    },
   }

In [12]:
s_orders = orders_api.search_orders(body).body['orders']

len(s_orders)

845

In [13]:
ls=[x['line_items'] for x in s_orders]

a=[x for x in ls if 'atch' not in x[0]['name']]
b=[x for x in a if 'Reg' not in x[0]['name']]
b=[x for x in b if '23' not in x[0]['name']]
b
# a
# ls

[[{'uid': '452e1790-a81f-4f0d-878c-90cb5a2a9ffc',
   'catalog_object_id': 'XR6BOAZK2WDAQTY3TYE5OGZK',
   'catalog_version': 1695059770548,
   'quantity': '1',
   'name': 'Yellow Card',
   'variation_name': 'Regular',
   'base_price_money': {'amount': 1200, 'currency': 'GBP'},
   'gross_sales_money': {'amount': 1200, 'currency': 'GBP'},
   'total_tax_money': {'amount': 0, 'currency': 'GBP'},
   'total_discount_money': {'amount': 0, 'currency': 'GBP'},
   'total_money': {'amount': 1200, 'currency': 'GBP'},
   'variation_total_price_money': {'amount': 1200, 'currency': 'GBP'},
   'item_type': 'ITEM',
   'total_service_charge_money': {'amount': 0, 'currency': 'GBP'}}],
 [{'uid': 'MQ0CQQeS1WqvKDLckQVvf',
   'quantity': '1',
   'name': 'Training',
   'base_price_money': {'amount': 500, 'currency': 'GBP'},
   'gross_sales_money': {'amount': 500, 'currency': 'GBP'},
   'total_tax_money': {'amount': 0, 'currency': 'GBP'},
   'total_discount_money': {'amount': 0, 'currency': 'GBP'},
   'total_mo

#  Get All Invoices

In [27]:
invoices_api = client.invoices


result = invoices_api.list_invoices(
    'SBBJSGR24YHYB'
)

invoices = pd.DataFrame(result.body['invoices'])

i=1
while result.cursor:
    print(i)
    i+=1
    result = invoices_api.list_invoices(
        'SBBJSGR24YHYB',
        cursor = result.cursor
    )
    print(result.cursor)
invoices = pd.concat([invoices, pd.DataFrame(result.body['invoices'])])
invoices


1
CgUIgwcYAQonCIMPEiJpbnY6MC1DaERGWHJIb0Rwc09RbFhWRE1Dd0Yyc3JFSU1Q
2
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEEza0RVUTIzWGtEZVdINGFGdE5qMnNFSU1Q
3
CgUIgwcYAQonCIMPEiJpbnY6MC1DaEN4VG0xVzg3R3hsRnVyeTZ5ZDhwVEdFSU1Q
4
CgUIgwcYAQonCIMPEiJpbnY6MC1DaENHVlQ2enNBdVlodDlUTm9JcVNYWkhFSU1Q
5
None


,id,version,location_id,order_id,payment_requests,primary_recipient,invoice_number,title,description,public_url,status,timezone,created_at,updated_at,accepted_payment_methods,delivery_method,sale_or_service_date,store_payment_method_enabled,next_payment_amount_money,scheduled_at
0,inv:0-ChD5h-2m_QTvCuvG_AitWgN6EIMP,3,SBBJSGR24YHYB,7cBxdecP1yTknMG7keX6lyjBucNZY,[{'uid': '359b7dd1-4e48-49b3-9daa-732a0848c2cc...,"{'customer_id': 'X3WGSQHHC97PK2KJZ91EBA9HNG', ...",2023-10-23_5_7cBxdecP1yTknMG7keX6lyjBucNZY_1,Match Subs 21 Oct vs Old Tiffonians,Match Subs 21 Oct vs Old Tiffonians,https://squareup.com/pay-invoice/inv:0-ChD5h-2...,PAID,Europe/London,2023-10-23T13:48:14Z,2023-10-23T15:57:44Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-10-23,False,NaN,NaN
1,inv:0-ChDaOMttk6_XZ62msO3Tgx1ZEIMP,3,SBBJSGR24YHYB,ZDtfsymRWrxR2L2fIfb6Ln1HFQJZY,[{'uid': 'e17b8d3d-95f1-4c2d-b520-69ba101ac264...,"{'customer_id': '1QVY1RMAN1734MNZB69GM4ZH0M', ...",2023-10-23_5_ZDtfsymRWrxR2L2fIfb6Ln1HFQJZY_1,Match Subs 21 Oct vs Old Tiffonians,Match Subs 21 Oct vs Old Tiffonians,https://squareup.com/pay-invoice/inv:0-ChDaOMt...,PAID,Europe/London,2023-10-23T13:48:14Z,2023-10-23T15:21:15Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-10-23,False,NaN,NaN
2,inv:0-ChAn0zyuiQ_qOmFNNkeDFmyyEIMP,2,SBBJSGR24YHYB,5bXE5wUP1NkbDo4b2rcioiM8CSIZY,[{'uid': '9a18b277-3a22-411d-a722-01dfc1a2cbc9...,"{'customer_id': '5BSGH6PJDW7PA17FQXZ03HQG74', ...",2023-10-23_5_5bXE5wUP1NkbDo4b2rcioiM8CSIZY_1,Match Subs 21 Oct vs Old Tiffonians,Match Subs 21 Oct vs Old Tiffonians,https://squareup.com/pay-invoice/inv:0-ChAn0zy...,UNPAID,Europe/London,2023-10-23T13:48:11Z,2023-10-23T15:04:23Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-10-23,False,"{'amount': 1000, 'currency': 'GBP'}",NaN
3,inv:0-ChAM6aowSAzEtZUmXHfHeY5rEIMP,3,SBBJSGR24YHYB,X863uvVeE9Y0V0tx4Qtl3pAhibJZY,[{'uid': '756a1b18-0f84-44da-98b5-7777d08679ab...,"{'customer_id': '4W39TT16K95QJXCB1Q9DGSDYW4', ...",2023-10-23_5_X863uvVeE9Y0V0tx4Qtl3pAhibJZY_1,Match Subs 21 Oct vs Old Tiffonians,Match Subs 21 Oct vs Old Tiffonians,https://squareup.com/pay-invoice/inv:0-ChAM6ao...,PAID,Europe/London,2023-10-23T13:48:11Z,2023-10-23T18:19:24Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-10-23,False,NaN,NaN
4,inv:0-ChDGWstpKKCdjI-pTiWArWl9EIMP,2,SBBJSGR24YHYB,ji8c4dDqgfKreY6fQ2V77valnkGZY,[{'uid': 'f1050585-16c0-47ca-820c-8d20118a165f...,"{'customer_id': '8QFE51ZKY13C5D4EATKWYFFCDW', ...",2023-10-23_5_ji8c4dDqgfKreY6fQ2V77valnkGZY_1,Match Subs 21 Oct vs Old Tiffonians,Match Subs 21 Oct vs Old Tiffonians,https://squareup.com/pay-invoice/inv:0-ChDGWst...,UNPAID,Europe/London,2023-10-23T13:48:10Z,2023-10-23T15:04:23Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-10-23,False,"{'amount': 1000, 'currency': 'GBP'}",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,inv:0-ChDPM4kl4NAiP5pXozC1Y_ESEIMP,1,SBBJSGR24YHYB,NaN,[{'uid': '35e32c13-7cdc-449f-b072-478fd634b433...,"{'customer_id': 'GB6X1PRCQZHR7BW76TTDCADACW', ...",000052,,,https://squareup.com/pay-invoice/inv:0-ChDPM4k...,UNPAID,Europe/London,2023-06-29T11:10:01Z,2023-07-03T06:51:24Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,2023-03-30,False,"{'amount': 500, 'currency': 'GBP'}",NaN
21,inv:0-ChAUAlcEksn2SoFvemS9qv1AEIMP,0,SBBJSGR24YHYB,NaN,[{'uid': '1bfd6dd0-1452-4d9e-905c-556bc79f0ba8...,"{'customer_id': 'DVXQMTVG4MFTHQ765VFCSDCZC8', ...",000053,,,https://squareup.com/pay-invoice/inv:0-ChAUAlc...,UNPAID,Europe/London,2023-06-29T11:44:40Z,2023-07-31T18:13:08Z,"{'card': True, 'square_gift_card': False, 'ban...",EMAIL,NaN,False,"{'amount': 500, 'currency': 'GBP'}",NaN
22,inv:0-ChBb4hlJg_JLA3LSygeItKZ3EIMP,1,SBBJSGR24YHYB,NaN,[{'uid': '20c3dd06-10de-444b-82e3-0b20104b9048...,"{'customer_id': 'X2KSKJ4GYN0M7G01D6SYKT1BHC', ...",000048,,,https://squareup.com/pay-invoice/inv:0-ChBb4hl...,UNPAID,Europe/London,2023-06-29T10:59:36Z,2023-07-06T12:04:54Z,"{'card': Tru

In [48]:
body = {
    'query': {
        'filter': {
            'location_ids': [
                'SBBJSGR24YHYB'
            ],
            'customer_ids': [
                'RX8HM0622ZC9YFTRCNW4Y5PPNG'
            ]
        },
        'sort': {
            'field': 'INVOICE_SORT_DATE',
            'order': 'DESC'
        }
    }
}

In [51]:
result = invoices_api.search_invoices(body)
for inv in result.body['invoices']:
    print(inv)

{'id': 'inv:0-ChD9FQVXiXJYAWFhjhdE_PdrEIMP', 'version': 3, 'location_id': 'SBBJSGR24YHYB', 'order_id': 'x3nItXZo9Gj8iJiYPwwpkFoDrQFZY', 'payment_requests': [{'uid': 'cb3bf115-f9a7-4535-9aed-3c7fba8d2510', 'request_type': 'BALANCE', 'due_date': '2023-10-17', 'tipping_enabled': True, 'computed_amount_money': {'amount': 1000, 'currency': 'GBP'}, 'total_completed_amount_money': {'amount': 1000, 'currency': 'GBP'}, 'reminders': [{'uid': '1a1e4e22-cda7-4119-bc92-c2ce826ee975', 'relative_scheduled_days': -1, 'message': 'Your Invoice is due', 'status': 'NOT_APPLICABLE'}], 'automatic_payment_source': 'NONE'}], 'primary_recipient': {'customer_id': 'RX8HM0622ZC9YFTRCNW4Y5PPNG', 'given_name': 'Ben', 'family_name': 'Dockerill', 'email_address': 'benwd@live.fr', 'phone_number': '+447592518802'}, 'invoice_number': '2023-10-17_1_x3nItXZo9Gj8iJiYPwwpkFoDrQFZY_1', 'title': 'Match Subs 14 Oct vs Manorians', 'description': 'Match Subs 14 Oct vs Manorians', 'public_url': 'https://squareup.com/pay-invoice/i

# Format new subs and get customer names

In [16]:
orders_format = []
for i, order in enumerate(s_orders[:]):
    
    print(i, '/', len(s_orders))
    new_order = {}
    
    new_order['id']=get_value(order,'id')
    new_order['created_at']=get_value(order,'updated_at')
    
    new_order['customer_id']=get_value(order,'customer_id')
    
    new_order['state']=get_value(order,'state')

    new_order['tenders']=get_value(order,'tenders')

    if not new_order['customer_id']:
        customer_ids = []
        try:
            for tender in order['tenders']:
                if get_value(tender,'customer_id'):
                    customer_ids.append(get_value(tender,'customer_id'))
            new_order['customer_id'] = customer_ids[0]
        except:
            pass

    if new_order['customer_id']:
        customer = customers_api.retrieve_customer(new_order['customer_id'])
    
        try:
            new_order['first_name'] = get_value(customer.body['customer'],'given_name')
            new_order['last_name'] = get_value(customer.body['customer'],'family_name')
        except:
            print(customer.body)
    else:
        pass
        invoice = invoices[invoices['order_id']==new_order['id']]
        
        
        if len(invoice)>0:
            cust_id = list(invoice['primary_recipient'])[0]['customer_id']
            customer = customers_api.retrieve_customer(cust_id)
            try:
                new_order['first_name'] = get_value(customer.body['customer'],'given_name')
                new_order['last_name'] = get_value(customer.body['customer'],'family_name')
            except:
                print(customer.body)
            
    for item in order['line_items']:
        line_dict = new_order.copy()
        
        line_dict['variation_name']=get_value(item,'variation_name')
        line_dict['item_name']=get_value(item,'name')
        if 'raining' in line_dict['item_name']:
            print(item)
        line_dict['note']=get_value(item,'note')
        line_dict['total_money']=item['total_money']['amount']/100
        orders_format.append(line_dict) 


orders_df = pd.DataFrame(orders_format)

0 / 845
1 / 845
2 / 845
3 / 845
4 / 845
5 / 845
6 / 845
7 / 845
8 / 845
9 / 845
10 / 845
11 / 845
12 / 845
13 / 845
14 / 845
15 / 845
16 / 845
17 / 845
18 / 845
19 / 845
20 / 845
21 / 845
22 / 845
23 / 845
24 / 845
25 / 845
26 / 845
27 / 845
28 / 845
29 / 845
30 / 845
31 / 845
32 / 845
33 / 845
34 / 845
35 / 845
36 / 845
37 / 845
38 / 845
39 / 845
40 / 845
41 / 845
42 / 845
43 / 845
44 / 845
{'errors': [{'code': 'NOT_FOUND', 'detail': 'Customer with ID `70740003A7AAN365QATYNCRFDG` not found.', 'category': 'INVALID_REQUEST_ERROR'}]}
45 / 845
46 / 845
47 / 845
48 / 845
49 / 845
50 / 845
51 / 845
52 / 845
53 / 845
{'uid': 'MQ0CQQeS1WqvKDLckQVvf', 'quantity': '1', 'name': 'Training', 'base_price_money': {'amount': 500, 'currency': 'GBP'}, 'gross_sales_money': {'amount': 500, 'currency': 'GBP'}, 'total_tax_money': {'amount': 0, 'currency': 'GBP'}, 'total_discount_money': {'amount': 0, 'currency': 'GBP'}, 'total_money': {'amount': 500, 'currency': 'GBP'}, 'variation_total_price_money': {'amo

In [47]:
orders_df[orders_df['id']=='1eU2NJ3TZ4z3WD8lwQAlj5feV']

,id,created_at,customer_id,state,tenders,first_name,last_name,variation_name,item_name,note,total_money
105,1eU2NJ3TZ4z3WD8lwQAlj5feV,2023-10-19T10:06:28.000Z,RX8HM0622ZC9YFTRCNW4Y5PPNG,COMPLETED,"[{'id': 'BUwVJ2bBCruKE2mdHsEW5UBQvaB', 'locati...",Ben,Dockerill,Regular,Yellow Card,None,12.0


In [23]:
list(orders_df[orders_df['item_name']=='Training']['tenders'])[0]

[{'id': 'ndZg5btuErH3BXcswsPk4nB35aHZY',
  'location_id': 'SBBJSGR24YHYB',
  'transaction_id': 'ToRlDWN1ThTb9IFQXqbF0tIEZSWZY',
  'created_at': '2023-09-10T19:10:32Z',
  'amount_money': {'amount': 500, 'currency': 'GBP'},
  'type': 'CARD',
  'card_details': {'status': 'CAPTURED',
   'card': {'card_brand': 'VISA',
    'last_4': '4154',
    'fingerprint': 'sq-1-z-eVQ6MM2t_xni0qmxPQqSf0RQlAKVDQ8E0MkJV3AUOlDGnlYcMgUg5_LLMmtsZFkg'},
   'entry_method': 'KEYED'},
  'tip_money': {'amount': 0, 'currency': 'GBP'},
  'payment_id': 'ndZg5btuErH3BXcswsPk4nB35aHZY'}]

# Get Correct columns and filter out unwanted orders

In [12]:
orders_df['customer_name']= orders_df['first_name'] + ' ' + orders_df['last_name']
match_fees_export = orders_df[(orders_df['state'].isin(['COMPLETED','OPEN'])) & (orders_df['tenders'].notnull())][['id', 'created_at', 'variation_name','item_name', 'customer_name', 'total_money', 'note']]
# match_fees_export = match_fees_export[match_fees_export['tenders'].notnull()]


match_fees_export = match_fees_export[(match_fees_export['customer_name'].notnull()) | (match_fees_export['note'].notnull())]


match_fees_export = match_fees_export[(match_fees_export['item_name'].fillna('').str.lower().str.contains('sub'))|(match_fees_export['item_name'].fillna('').str.lower().str.contains('training'))]

In [13]:
match_fees_export.columns = exist.columns

In [14]:
match_fees_export

,ID,Date,Cat,Item,Customer,Price,Notes
0,L8jU3V7OqgFJeWtNdihvOo7JPcBZY,2023-09-04T13:42:27.000Z,Full Game,Match Sub,Adam Platten,5.0,None
2,ZLfm2FvJNVRvMfdkRsl2SbE1OOPZY,2023-09-04T11:25:25.000Z,Full Game,Match Sub,Denver Lalonde,10.0,None


# Only upload new transactions

In [15]:
mg = pd.merge(left=match_fees_export, right=exist, how='left', on='ID', suffixes=['', '_old'])
not_in = mg[mg['Date_old'].isnull()][exist.columns]
len(not_in)

1

In [16]:
not_in

,ID,Date,Cat,Item,Customer,Price,Notes
0,L8jU3V7OqgFJeWtNdihvOo7JPcBZY,2023-09-04T13:42:27.000Z,Full Game,Match Sub,Adam Platten,5.0,None


In [17]:
not_in.fillna('NA', inplace=True)

# Add to Google sheet

In [18]:
append_to_sheet('Square Subs', SAMPLE_SPREADSHEET_ID_input, not_in)